**Download Data**

In [ ]:
!gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
!unzip data.zip
!ls 

Downloading...
From: https://drive.google.com/uc?id=1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR
To: /content/data.zip
372MB [00:01, 221MB/s]
Archive:  data.zip
   creating: timit_11/
  inflating: timit_11/train_11.npy   
  inflating: timit_11/test_11.npy    
  inflating: timit_11/train_label_11.npy  
data.zip  sample_data  timit_11


**Preparing Data**

In [ ]:
import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


**Create Dataset**

In [ ]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
  def __init__(self, X, y = None):
    self.data = torch.from_numpy(X).float()
    if y is not None:
      y = y.astype(np.int)
      self.label = torch.LongTensor(y)
    else:
      self.label = None
    
  def __getitem__(self, idx):
    if self.label is not None:
      return self.data[idx], self.label[idx]
    else:
      return self.data[idx]

  def __len__(self):
    return len(self.data)


In [ ]:
VAL_RATIO = 0.2

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (983945, 429)
Size of validation set: (245987, 429)


In [ ]:
BATCH_SIZE = 64

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x,train_y)
val_set = TIMITDataset(val_x,val_y)
train_loader = DataLoader(train_set, batch_size = BATCH_SIZE, shuffle = True)
val_loader = DataLoader(val_set, batch_size = BATCH_SIZE, shuffle = False)

In [ ]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

153

**Create Model**

In [ ]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
  def __init__(self):
    super(Classifier, self).__init__()
    self.layer1 = nn.Linear(429, 1024)
    self.layer2 = nn.Linear(1024, 512)
    self.layer3 = nn.Linear(512, 128)
    self.out = nn.Linear(128, 39)

    self.act_fn = nn.Sigmoid()
  def forward(self, x):
    x = self.layer1(x)
    x = self.act_fn(x)

    x = self.layer2(x)
    x = self.act_fn(x)

    x = self.layer3(x)
    x = self.act_fn(x)

    x = self.out(x)
        
    return x

**Training**

In [ ]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

**Fix random seeds for reproducibility**

In [ ]:
#fix random seed
def same_seeds(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)
  torch.backends.cudnn.benchmark = False
  torch.backends.cudnn.deterministic = True

**Feel free to change the training parameters here**

In [ ]:
#fix random seed for reproducibility
same_seeds(0)

#get device
device = get_device()
print(f'Device: {device}')

#training parameters
num_epoch = 20
learning_rate = 0.0001

#the path where checkpoint saved
model_path = './model.ckpt'

#create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

Device: cuda


In [ ]:
#start training

best_acc = 0.0
for epoch in range(num_epoch):
  train_acc = 0.0
  train_loss = 0.0
  val_acc = 0.0
  val_loss = 0.0

  #training
  model.train()
  for i, data in enumerate(train_loader):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(inputs)
    batch_loss = criterion(outputs, labels)
    _, train_pred = torch.max(outputs, 1)
    batch_loss.backward()
    optimizer.step()

    train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
    train_loss += batch_loss.item()
  if len(val_set) > 0:
    model.eval()
    with torch.no_grad():
      for i, data in enumerate(val_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        batch_loss = criterion(outputs, labels)
        _, val_pred = torch.max(outputs, 1)
        
        val_acc += (val_pred.cpu() == labels.cpu()).sum().item()
        val_loss += batch_loss.item()
      print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))
      
      #if model improves, save a checkpoint at this epoch
      if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), model_path)
        print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
  else:
    print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))
if len(val_set) == 0:
  torch.save(model.state_dict(), model_path)
  print('saving model at last epoch')

[001/020] Train Acc: 0.467302 Loss: 1.811661 | Val Acc: 0.567428 loss: 1.433065
saving model with acc 0.567
[002/020] Train Acc: 0.594383 Loss: 1.330666 | Val Acc: 0.628639 loss: 1.211098
saving model with acc 0.629
[003/020] Train Acc: 0.644506 Loss: 1.154064 | Val Acc: 0.660421 loss: 1.101216
saving model with acc 0.660
[004/020] Train Acc: 0.672217 Loss: 1.052246 | Val Acc: 0.676300 loss: 1.038718
saving model with acc 0.676
[005/020] Train Acc: 0.691347 Loss: 0.983104 | Val Acc: 0.685154 loss: 1.001852
saving model with acc 0.685
[006/020] Train Acc: 0.705615 Loss: 0.931955 | Val Acc: 0.689301 loss: 0.984177
saving model with acc 0.689
[007/020] Train Acc: 0.716344 Loss: 0.891687 | Val Acc: 0.694516 loss: 0.964627
saving model with acc 0.695
[008/020] Train Acc: 0.725881 Loss: 0.857907 | Val Acc: 0.697720 loss: 0.951889
saving model with acc 0.698
[009/020] Train Acc: 0.733718 Loss: 0.829495 | Val Acc: 0.696691 loss: 0.949866
[010/020] Train Acc: 0.741152 Loss: 0.803701 | Val Acc: 

**Testing**

In [ ]:
#create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size = BATCH_SIZE, shuffle = False)

#create model and load weights from checkpoints
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

**Make prediction**

In [ ]:
predict = []
model.eval()
with torch.no_grad():
  for i, data in enumerate(test_loader):
    inputs = data
    inputs = inputs.to(device)
    outputs = model(inputs)
    _, test_pred = torch.max(outputs, 1)

    for y in test_pred.cpu().numpy():
      predict.append(y)

**Write prediction to a CSV file**

In [ ]:
with open('prediction.csv', 'w') as f:
  f.write('Id,Class\n')
  for i, y in enumerate(predict):
    f.write('{},{}\n'.format(i, y))